In [15]:
#only for testing
def return_question():
    import random
    next_ques=random.choice(["Yooooo!",'Heyyyy!','Weeeee!'])
    pic_link='https://bakingamoment.com/wp-content/uploads/2018/02/IMG_6207-chocolate-molten-lava-cakes-recipe-square.jpg'
    return next_ques,pic_link


from tkinter import *
from tkinter import scrolledtext

#functions to be implemented: connect each page to the randomly generated question
#user can choose from a multiple choice question
#user is told right or wrong
#count the correct answer number
#quit button
class quizGUI:
    
    def __init__(self):
        root = Tk()
        root.lift()
        root.attributes("-topmost", True)
        root.geometry("500x600")
        root.title("MET-Q")
        self.root = root
        
        #correct/incorrect indicator
        var=StringVar()
        self.cor_label=Label(root,textvariable=var)
        self.var=var
        self.cor_label.pack()
        #title label
        self.title_label = Label(root, text="The MET Quiz", font = "Times 20 bold")
        self.title_label.pack(side="top")
    
        #record the number of correct answers so far
        self.total = IntVar() 
        self.total_var=StringVar()
        self.total_var.set('Number of correct answers: 0')
        self.score_label = Label(root,textvariable=self.total_var)
        self.score_label.pack()
        #record the number of questions completed so far
        self.num_var=StringVar()
        self.num=IntVar()
        self.num_var.set('Questions completed: 0')
        self.num_of_ques=Label(root,textvariable=self.num_var)
        self.num_of_ques.pack()
    
        #question stem
        question_var=StringVar()
        self.Q = Label(root, textvariable=question_var)
        question_var.set(return_question())
        self.Q.pack()
        self.question_var=question_var
        
        
        
        #Question picture
        pic=tk.PhotoImage()
        
        
        
        
        #choice buttons
        self.A = Button(root, text="A",command=lambda: self.correct()).pack()
        self.B = Button(root, text="B",command=lambda: self.incorrect()).pack()
        self.C = Button(root, text="C",command=lambda: self.incorrect()).pack()
        self.D = Button(root, text="D",command=lambda: self.incorrect()).pack()
        self.next=Button(root,text='Next question',command=lambda:self.next_()).pack()

        #quit button
        self.close_button=Button(root, text="Quit", command=lambda: root.destroy())
        self.close_button.pack(padx=20,side='right')
          
        root.mainloop() 
        

    #add 1 to score if the correct answer is chosen
    def correct(self):
        self.var.set('Correct')
        self.total.set(self.total.get() + 1) 
        self.total_var.set('Number of correct answers: '+str(self.total.get()))
    
    def incorrect(self):
        self.var.set('Incorrect')

        
    def next_(self):
        self.question_var.set(return_question()[0])
        self.num.set(self.num.get()+1)
        self.num_var.set('Questions completed: '+str(self.num.get()))

       

In [16]:
myGUI = quizGUI()